## Using ANN on TitanicSurvivalData

In [1]:
# Reading the cleaned numeric car prices data
import pandas as pd
import numpy as np

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

TitanicSurvivalDataNumeric=pd.read_pickle('TitanicSurvivalDataNumeric.pkl')
TitanicSurvivalDataNumeric.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [2]:
TitanicSurvivalDataNumeric.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

## Splitting the Data into Training and Testing

In [3]:
# After Standardization of data
# Separate Target Variable and Predictor Variables
TargetVariable=['Survived']
Predictors=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked_C', 'Embarked_Q', 'Embarked_S']

X=TitanicSurvivalDataNumeric[Predictors].values
y=TitanicSurvivalDataNumeric[TargetVariable].values


### Sandardization of data ###
### We does not standardize the Target variable for classification
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(623, 9)
(623, 1)
(268, 9)
(268, 1)


## Classification using Deep Learning- Artificial Neural Networks(ANN)

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
classifier = Sequential()
# Defining the Input layer and FIRST hidden layer
# relu means Rectifier linear unit function
classifier.add(Dense(units=10, input_dim=9, kernel_initializer='uniform', activation='relu'))

#Defining the SECOND hidden layer, here we have not defined input because it is
# second layer and it will get input as the output of first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Defining the Output layer
# sigmoid means sigmoid activation function
# for Multiclass classification the activation ='softmax'
# And output_dim will be equal to the number of factor levels
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Optimizer== the algorithm of SGG to keep updating weights
# loss== the loss function to measure the accuracy
# metrics== the way we will compare the accuracy after each step of SGD
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
# fitting the Neural Network on the training data
survivalANN_Model=classifier.fit(X_train,y_train, batch_size=10 , epochs=10, verbose=1)

Epoch 1/10
623/623 [==============================] - 1s - loss: 0.6895 - acc: 0.6308     
Epoch 2/10
623/623 [==============================] - 0s - loss: 0.6714 - acc: 0.6661     
Epoch 3/10
623/623 [==============================] - 0s - loss: 0.6105 - acc: 0.7849     
Epoch 4/10
623/623 [==============================] - 0s - loss: 0.5302 - acc: 0.8090     
Epoch 5/10
623/623 [==============================] - 0s - loss: 0.4793 - acc: 0.8026     
Epoch 6/10
623/623 [==============================] - 0s - loss: 0.4612 - acc: 0.8058     
Epoch 7/10
623/623 [==============================] - 0s - loss: 0.4536 - acc: 0.8122     
Epoch 8/10
623/623 [==============================] - 0s - loss: 0.4488 - acc: 0.8058     
Epoch 9/10
623/623 [==============================] - 0s - loss: 0.4442 - acc: 0.8090     
Epoch 10/10
623/623 [==============================] - 0s - loss: 0.4420 - acc: 0.8058     


In [13]:
# Training data Accuracy of the model is the the accuracy achieved in the last epoch
survivalANN_Model.history['acc'][-1]

0.8057784887320731

## Finding best set of parameters using grid search

In [121]:
def FunctionFindBestParams(X_train, y_train):
    
    
    # Defining the list of hyper parameters to try
    TrialNumber=0
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50 ,100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            # Creating the classifier ANN model
            classifier = Sequential()
            classifier.add(Dense(units=10, input_dim=9, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
            classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            
            survivalANN_Model=classifier.fit(X_train,y_train, batch_size=batch_size_trial , epochs=epochs_trial, verbose=0)
            Accuracy = survivalANN_Model.history['acc'][-1]
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', Accuracy)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber,
                            'batch_size'+str(batch_size_trial)+'-'+'epoch'+str(epochs_trial), Accuracy]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)

In [122]:
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train)

1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 0.8121990464376599
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 0.8105939105751043
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 0.8426966373696565
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 0.8475120466555294
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 0.7961476708300423
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 0.8089887606006947
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 0.8330658066712834
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 0.8410914859840614
9 Parameters: batch_size: 15 - epochs: 5 Accuracy: 0.8138041843571977
10 Parameters: batch_size: 15 - epochs: 10 Accuracy: 0.8073836383716253
11 Parameters: batch_size: 15 - epochs: 50 Accuracy: 0.8378812294710505
12 Parameters: batch_size: 15 - epochs: 100 Accuracy: 0.8394863657163004
13 Parameters: batch_size: 20 - epochs: 5 Accuracy: 0.629213487403542
14 Parameters: batch_size: 20 - epochs: 10 Accuracy: 0.6292134865903165
15 Para

### Training the model using best parameters

In [138]:
classifier.fit(X_train,y_train, batch_size=5 , epochs=100, verbose=1)

Epoch 1/100
623/623 [==============================] - 0s - loss: 0.4185 - acc: 0.8347     
Epoch 2/100
623/623 [==============================] - 0s - loss: 0.4135 - acc: 0.8315     
Epoch 3/100
623/623 [==============================] - 0s - loss: 0.4152 - acc: 0.8347     
Epoch 4/100
623/623 [==============================] - 0s - loss: 0.4147 - acc: 0.8315     
Epoch 5/100
623/623 [==============================] - 0s - loss: 0.4143 - acc: 0.8347     
Epoch 6/100
623/623 [==============================] - 0s - loss: 0.4141 - acc: 0.8331     
Epoch 7/100
623/623 [==============================] - 0s - loss: 0.4099 - acc: 0.8363     
Epoch 8/100
623/623 [==============================] - 0s - loss: 0.4118 - acc: 0.8363     
Epoch 9/100
623/623 [==============================] - 0s - loss: 0.4133 - acc: 0.8331     
Epoch 10/100
623/623 [==============================] - 0s - loss: 0.4145 - acc: 0.8363     
Epoch 11/100
623/623 [==============================] - 0s - loss: 0.4111 - acc

### Generating the predictions on Testing Data

In [139]:
# Predictions on testing data
Predictions=classifier.predict(X_test)

# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

# Generating a data frame for analyzing the test data
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Survival']=y_test
TestingData['PredictedSurvivalProb']=Predictions

def probThreshold(inpProb):
    if inpProb>0.5:
        return(1)
    else:
        return(0)

TestingData['PredictedSurvival']=TestingData['PredictedSurvivalProb'].apply(probThreshold)
TestingData.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Survival,PredictedSurvivalProb,PredictedSurvival
0,3.0,1.0,23.0,1.0,1.0,15.2458,1.0,0.0,0.0,1,0.249054,0
1,2.0,1.0,31.0,0.0,0.0,10.5000,0.0,0.0,1.0,0,0.161471,0
2,3.0,1.0,20.0,0.0,0.0,7.9250,0.0,0.0,1.0,0,0.094225,0
3,2.0,0.0,6.0,0.0,1.0,33.0000,0.0,0.0,1.0,1,0.791084,1
4,3.0,0.0,14.0,1.0,0.0,11.2417,1.0,0.0,0.0,1,0.791084,1


In [140]:
# Comparing a sample row from original data to confirm if the reverse transform has happend correctly
TitanicSurvivalDataNumeric[(TitanicSurvivalDataNumeric['Fare']==11.2417) & (TitanicSurvivalDataNumeric['Age']==14)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
39,1,3,0,14.0,1,0,11.2417,1,0,0


### Calculating the Accuracy on Testing Data

In [141]:
from sklearn import metrics
print(metrics.classification_report(TestingData['Survival'], TestingData['PredictedSurvival']))
print(metrics.confusion_matrix(TestingData['Survival'], TestingData['PredictedSurvival']))

             precision    recall  f1-score   support

          0       0.82      0.87      0.84       157
          1       0.80      0.74      0.77       111

avg / total       0.81      0.81      0.81       268

[[136  21]
 [ 29  82]]
